In [1]:
import os
import re
import json

import numpy as np
import pandas as pd

from collections import OrderedDict, defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.SeqUtils.IsoelectricPoint import IsoelectricPoint as IP

**Data:**

In [2]:
meta_dir = 'metadata'
upstreams_dir = 'upstream_search'
res_dir = 'results'

hosts_file = os.path.join(meta_dir, 'seq_id_to_host.tsv')
faa_file = os.path.join(upstreams_dir, 'upstream.faa')
gff_file = os.path.join(res_dir, 'upstreams_clusters_domains.gff')
res_file = os.path.join(res_dir, 'upstream_proteins_clu_wide_seq_sorted_prokka_domains.tsv')

output_file = os.path.join(res_dir, 'upstream_proteins_clu_wide_seq_sorted_prokka_domains_pi_length_host.tsv')

**Read info prior information about clusters**:

In [3]:
res_table = pd.read_csv(res_file, sep='\t')
res_table['cluster_num'] = res_table.index
res_table

,clu_parent,count,products,sequence,phrOG,phrOG_annotation,phrOG_category,pfam_annotation,pfam,conj_pl_domains,cdd,cdd_name,tigr,tigrfam_name,cluster_num
1,GCF_001470995.1_00023,354,{T7 RNA polymerase: 354},MEYNPLTELASLYGEDLAAEQLRLEAEAYSLGEKRFMEAMEFKAET...,"phrog_20599,phrog_414",RNA polymerase,"DNA, RNA and nucleotide metabolism","RNA_pol,RPOL_N","PF00940.22,PF14700.9",NaN,"cl42978,cl34907,cl44477,cl20638","PHA00452,RPO41,RNA_pol,RPOL_N",NaN,NaN,1
2,GCA_025085155.1_00014,190,"{Protein kinase 0.7: 183, hypothetical protein...",MYAHFQSLLKAIRELPINELDKRQPMLVDLLAQIVNHETQDGAITA...,"phrog_18305,phrog_2828,phrog_15752","kinase,serine-threonine kinase,protein kinase;...",other,NaN,NaN,NaN,cl20113,PHA00451,NaN,NaN,2
3,GCF_001041815.1_00006,150,"{Overcome classical restriction gp0.3: 102, hy...",MSNVTYHSLYLSAKQGLINVIRDNDLRDYDEACDYVHEVADIYVPH...,phrog_2568,ocr-like anti-restriction,"moron, auxiliary metabolic gene and host takeover",ocr,PF08684.13,NaN,NaN,NaN,NaN,NaN,3
4,GCA_023716895.1_00035,149,{hypothetical protein: 149},METSCRLPHGAGRSCTGMDGDAMMFTAFIVFMYLLIVLYFLKDFRK...,phrog_2385,NaN,unknown function,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
5,GCA_009672325.1_00007,144,{hypothetical protein: 144},MIYTNEPANIFYMYVSAARADRPDVVNLARHNQLKRDIMNGLGMYG...,phrog_2226,SAM-dependent methyltransferase,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,GCA_005892785.1_00005,1,{hypothetical protein: 1},MFKHEVFISDNREAAMIAEMFKGHVTAYNIGDDEEAFCRYLC,phrog_6403,NaN,unknown function,NaN,NaN,NaN,NaN,NaN,NaN,NaN,681
682,GCA_005892125.1_00006,1,{hypothetical protein: 1},MKYILIAMFSGFIGYAGASYDNYRSKQLAAEKWDKTYFACMKQGNL...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,682
683,GCA_005280775.1_00002,1,{hypothetical protein: 1},MVTYYVYVNQMEYFEADTLEEAIRLCDSVPGSHVEDSVLDEVIYANR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,683
684,GCA_002922415.1_00057,1,{Tail spike protein: 1},VNSSVGVGSVVVKDSYIYYIFGGENHFNPMTYGDNKDKDPFKGHGH...,"phrog_38424,phrog_17635,phrog_14895","tail fiber protein,tail spike protein","unknown function,tail","End_beta_propel,End_tail_spike,Peptidase_S74","PF12217.11,PF12219.11,PF13884.9",NaN,"cl13629,cl16452,cl28659","End_tail_spike,Peptidase_S74_CIMCD,End_beta_pr...",NaN,NaN,684


**Read information about hosts:**

In [4]:
hosts_data = pd.read_csv(hosts_file, sep='\t')

In [5]:
hosts_data.describe()
hosts_data['is_Escherichia'] = hosts_data['order'] == 'Escherichia'
hosts_data['is_Enterobacterales'] = hosts_data['order'] == 'Enterobacterales'

**Read GFF with information about domains, clusters and nuccore ids.**  
Take only nessesary columns:

In [6]:
colnames = ['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attribute']
gff_data = pd.read_csv(gff_file, sep='\t', skiprows=1, names=colnames)
gff_data = gff_data.drop(columns=['source', 'type', 'score', 'phase'])

Create a dict from attribute string:

In [7]:
gff_data['attribute'] = gff_data['attribute'].apply(lambda x: x.replace('=;', '=NaN;'))  # fix a data artefact
gff_data['attribute_dict'] = gff_data['attribute'].apply(lambda x: {i.split('=')[0]: 
                                                                    i.split('=')[1] for i in x.split(';')})
gff_data.head()

,seq_id,start,end,strand,attribute,attribute_dict
0,AY264775.1,901,1254,+,ID=GCA_002600005.1_00001;file_id=upstreams_wit...,"{'ID': 'GCA_002600005.1_00001', 'file_id': 'up..."
1,AY264775.1,1254,1409,+,ID=GCA_002600005.1_00002;file_id=upstreams_wit...,"{'ID': 'GCA_002600005.1_00002', 'file_id': 'up..."
2,AY264775.1,1478,1615,+,ID=GCA_002600005.1_00003;file_id=upstreams_wit...,"{'ID': 'GCA_002600005.1_00003', 'file_id': 'up..."
3,AY264775.1,1612,2256,+,ID=GCA_002600005.1_00004;file_id=upstreams_wit...,"{'ID': 'GCA_002600005.1_00004', 'file_id': 'up..."
4,AY264775.1,2327,4978,+,ID=GCA_002600005.1_00005;Name=1;file_id=upstre...,"{'ID': 'GCA_002600005.1_00005', 'Name': '1', '..."


Create separate columns from attribute dict:

In [8]:
norm_attribute = pd.json_normalize(gff_data.attribute_dict)

In [9]:
norm_attribute.head()

,ID,file_id,inference,locus_tag,product,cluster_num,cluster_main_prod,phrOG,phrOG_annotation,phrOG_category,pfam_annotation,pfam,e_c_number,Name,gene,cdd,cdd_name,tigr,tigrfam_name,conj_pl_domains
0,GCA_002600005.1_00001,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00001,Overcome classical restriction gp0.3,2,Overcome classical restriction gp0.3,phrog_2568,ocr-like anti-restriction,"moron, auxiliary metabolic gene and host takeover",ocr,PF08684.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GCA_002600005.1_00002,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00002,Gene 0.4 protein,12,Gene 0.4 protein,"phrog_25044,phrog_7202",NaN,unknown function,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GCA_002600005.1_00003,upstreams_with_clusters,ab initio prediction:Prodigal:2.6,GCA_002600005.1_00003,hypothetical protein,18,hypothetical protein,phrog_16572,NaN,unknown function,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GCA_002600005.1_00004,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00004,Protein kinase 0.7,134,protein kinase,"phrog_8393,phrog_29241,phrog_2828,phrog_15752","serine-threonine kinase,protein kinase, inhibi...","unknown function,other",NaN,NaN,2.7.11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GCA_002600005.1_00005,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00005,T7 RNA polymerase,0,RNA polymerase,"phrog_2051,phrog_2166,phrog_414","RNA polymerase,N4-like RNA polymerase","DNA, RNA and nucleotide metabolism","RNA_pol,RPOL_N","PF00940.22,PF14700.9",2.7.7.6,1,1,NaN,NaN,NaN,NaN,NaN


"Append" normalized attribute to gff dataframe:

In [10]:
gff_data = pd.concat([gff_data, norm_attribute], axis=1)
# remove temp columns:
gff_data = gff_data.drop(columns=['attribute', 'attribute_dict'])

In [11]:
gff_data.head()

,seq_id,start,end,strand,ID,file_id,inference,locus_tag,product,cluster_num,...,pfam_annotation,pfam,e_c_number,Name,gene,cdd,cdd_name,tigr,tigrfam_name,conj_pl_domains
0,AY264775.1,901,1254,+,GCA_002600005.1_00001,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00001,Overcome classical restriction gp0.3,2,...,ocr,PF08684.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AY264775.1,1254,1409,+,GCA_002600005.1_00002,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00002,Gene 0.4 protein,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AY264775.1,1478,1615,+,GCA_002600005.1_00003,upstreams_with_clusters,ab initio prediction:Prodigal:2.6,GCA_002600005.1_00003,hypothetical protein,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AY264775.1,1612,2256,+,GCA_002600005.1_00004,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00004,Protein kinase 0.7,134,...,NaN,NaN,2.7.11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AY264775.1,2327,4978,+,GCA_002600005.1_00005,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00005,T7 RNA polymerase,0,...,"RNA_pol,RPOL_N","PF00940.22,PF14700.9",2.7.7.6,1,1,NaN,NaN,NaN,NaN,NaN


In [12]:
gff_data.cluster_num.describe()  # check-in

count     3186
unique     685
top          0
freq       354
Name: cluster_num, dtype: object

**Read sequence data:**

In [13]:
with open(faa_file, 'r') as fasta_file:
    records = list(SimpleFastaParser(fasta_file))

prot_seq_df = pd.DataFrame(records, columns = ['ID', 'sequence'])
prot_seq_df.head()

,ID,sequence
0,GCA_002600005.1_00001,MAMSNMTYNNVFNHAYEMLKENIRYDDIRDTDDLHDAIHMTADNAV...
1,GCA_002600005.1_00002,MSTTNVQYGLTAQTVFFYSDMVRCGFNWSLAMAQLKELYENNKAIA...
2,GCA_002600005.1_00003,MLTIGLLTVLDLAVGASFGKALGVAVGSYFTACIIIEIIKGALRK
3,GCA_002600005.1_00004,MMKHYVMPIHTSNGTTVCTPDGFAMKQRIERLKRELRINRKFFEGI...
4,GCA_002600005.1_00005,MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEM...


In [14]:
prot_seq_df.describe()

,ID,sequence
count,3186,3186
unique,3186,2803
top,GCA_002600005.1_00001,MSKLLATSKIEGQCTVTLREYYHGSMGSTYVVRYGKQVTHWVNPIL...
freq,1,11


**Calculate sequence metrics: length, pI for each sequence.**

In [15]:
prot_seq_df['pI'] = prot_seq_df['sequence'].apply(lambda x: IP(x).pi())
prot_seq_df['pI'] = prot_seq_df['pI'].apply(lambda x: round(x, 2))
prot_seq_df['length'] = prot_seq_df['sequence'].apply(lambda x: len(x))

In [16]:
prot_seq_df.head()

,ID,sequence,pI,length
0,GCA_002600005.1_00001,MAMSNMTYNNVFNHAYEMLKENIRYDDIRDTDDLHDAIHMTADNAV...,4.05,117
1,GCA_002600005.1_00002,MSTTNVQYGLTAQTVFFYSDMVRCGFNWSLAMAQLKELYENNKAIA...,4.75,51
2,GCA_002600005.1_00003,MLTIGLLTVLDLAVGASFGKALGVAVGSYFTACIIIEIIKGALRK,9.10,45
3,GCA_002600005.1_00004,MMKHYVMPIHTSNGTTVCTPDGFAMKQRIERLKRELRINRKFFEGI...,10.32,214
4,GCA_002600005.1_00005,MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEM...,7.59,883


**Aggregate results.**

In [17]:
df = pd.concat([gff_data, prot_seq_df], axis=1)
df = df.merge(hosts_data, on='seq_id') 
df.head()

,seq_id,start,end,strand,ID,file_id,inference,locus_tag,product,cluster_num,...,tigr,tigrfam_name,conj_pl_domains,ID,sequence,pI,length,order,is_Escherichia,is_Enterobacterales
0,AY264775.1,901,1254,+,GCA_002600005.1_00001,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00001,Overcome classical restriction gp0.3,2,...,NaN,NaN,NaN,GCA_002600005.1_00001,MAMSNMTYNNVFNHAYEMLKENIRYDDIRDTDDLHDAIHMTADNAV...,4.05,117,Enterobacterales,False,True
1,AY264775.1,1254,1409,+,GCA_002600005.1_00002,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00002,Gene 0.4 protein,12,...,NaN,NaN,NaN,GCA_002600005.1_00002,MSTTNVQYGLTAQTVFFYSDMVRCGFNWSLAMAQLKELYENNKAIA...,4.75,51,Enterobacterales,False,True
2,AY264775.1,1478,1615,+,GCA_002600005.1_00003,upstreams_with_clusters,ab initio prediction:Prodigal:2.6,GCA_002600005.1_00003,hypothetical protein,18,...,NaN,NaN,NaN,GCA_002600005.1_00003,MLTIGLLTVLDLAVGASFGKALGVAVGSYFTACIIIEIIKGALRK,9.10,45,Enterobacterales,False,True
3,AY264775.1,1612,2256,+,GCA_002600005.1_00004,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00004,Protein kinase 0.7,134,...,NaN,NaN,NaN,GCA_002600005.1_00004,MMKHYVMPIHTSNGTTVCTPDGFAMKQRIERLKRELRINRKFFEGI...,10.32,214,Enterobacterales,False,True
4,AY264775.1,2327,4978,+,GCA_002600005.1_00005,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00005,T7 RNA polymerase,0,...,NaN,NaN,NaN,GCA_002600005.1_00005,MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEM...,7.59,883,Enterobacterales,False,True


In [18]:
df_agg = df.groupby('cluster_num').agg(
                        MedianpI = pd.NamedAgg(column = 'pI', aggfunc=np.median), 
                        LQpI = pd.NamedAgg(column = 'pI', aggfunc = lambda x: x.quantile(0.25)),
                        UQpI = pd.NamedAgg(column = 'pI', aggfunc = lambda x: x.quantile(0.75)),
                        MedianLength = pd.NamedAgg(column = 'length', aggfunc=np.median), 
                        LQLenths = pd.NamedAgg(column = 'length', aggfunc = lambda x: x.quantile(0.25)),
                        UQLenths = pd.NamedAgg(column = 'length', aggfunc = lambda x: x.quantile(0.75)),
                        Ecoli_host = pd.NamedAgg(column = 'is_Escherichia', 
                                                 aggfunc = lambda x: x.any()),
                        Enterobacter_host = pd.NamedAgg(column = 'is_Enterobacterales', 
                                                 aggfunc = lambda x: x.any())
                                )

In [19]:
df_agg.index = df_agg.index.astype(int)
df_agg.sort_index(inplace=True)

In [20]:
cluster_num = pd.Series(range(1, len(df_agg)+1))
df_agg.set_index(cluster_num, inplace=True)
df_agg['cluster_num'] = pd.Series(range(0, len(df_agg)+1))

In [21]:
df_agg.head()

,MedianpI,LQpI,UQpI,MedianLength,LQLenths,UQLenths,Ecoli_host,Enterobacter_host,cluster_num
1,6.960,6.7100,7.1500,888.0,883.0,906.0,True,True,1
2,6.650,6.2675,7.5225,359.0,337.0,369.0,True,True,2
3,4.050,4.0500,4.0500,117.0,114.0,121.0,True,True,3
4,10.780,10.5000,10.8200,65.0,65.0,65.0,True,True,4
5,5.515,5.2200,6.8200,152.0,152.0,155.0,True,True,5


**Merge results**

In [22]:
res_table = res_table.merge(df_agg, on='cluster_num', how='left')

In [23]:
res_table.head()

,clu_parent,count,products,sequence,phrOG,phrOG_annotation,phrOG_category,pfam_annotation,pfam,conj_pl_domains,...,tigrfam_name,cluster_num,MedianpI,LQpI,UQpI,MedianLength,LQLenths,UQLenths,Ecoli_host,Enterobacter_host
0,GCF_001470995.1_00023,354,{T7 RNA polymerase: 354},MEYNPLTELASLYGEDLAAEQLRLEAEAYSLGEKRFMEAMEFKAET...,"phrog_20599,phrog_414",RNA polymerase,"DNA, RNA and nucleotide metabolism","RNA_pol,RPOL_N","PF00940.22,PF14700.9",NaN,...,NaN,1,6.960,6.7100,7.1500,888.0,883.0,906.0,True,True
1,GCA_025085155.1_00014,190,"{Protein kinase 0.7: 183, hypothetical protein...",MYAHFQSLLKAIRELPINELDKRQPMLVDLLAQIVNHETQDGAITA...,"phrog_18305,phrog_2828,phrog_15752","kinase,serine-threonine kinase,protein kinase;...",other,NaN,NaN,NaN,...,NaN,2,6.650,6.2675,7.5225,359.0,337.0,369.0,True,True
2,GCF_001041815.1_00006,150,"{Overcome classical restriction gp0.3: 102, hy...",MSNVTYHSLYLSAKQGLINVIRDNDLRDYDEACDYVHEVADIYVPH...,phrog_2568,ocr-like anti-restriction,"moron, auxiliary metabolic gene and host takeover",ocr,PF08684.13,NaN,...,NaN,3,4.050,4.0500,4.0500,117.0,114.0,121.0,True,True
3,GCA_023716895.1_00035,149,{hypothetical protein: 149},METSCRLPHGAGRSCTGMDGDAMMFTAFIVFMYLLIVLYFLKDFRK...,phrog_2385,NaN,unknown function,NaN,NaN,NaN,...,NaN,4,10.780,10.5000,10.8200,65.0,65.0,65.0,True,True
4,GCA_009672325.1_00007,144,{hypothetical protein: 144},MIYTNEPANIFYMYVSAARADRPDVVNLARHNQLKRDIMNGLGMYG...,phrog_2226,SAM-dependent methyltransferase,other,NaN,NaN,NaN,...,NaN,5,5.515,5.2200,6.8200,152.0,152.0,155.0,True,True


In [24]:
last_seq = res_table.iloc[-1, 3]
last_pi = round(IP(last_seq).pi(), 2)
last_length = len(last_seq)
res_table.iloc[-1, 18] = last_length
res_table.iloc[-1, 15] = last_pi

In [25]:
# res_table.to_csv(output_file, sep='\t', index=False)

In [26]:
res_table.query('MedianpI < 6 and MedianLength > 100 and (Ecoli_host or Enterobacter_host)')

,clu_parent,count,products,sequence,phrOG,phrOG_annotation,phrOG_category,pfam_annotation,pfam,conj_pl_domains,...,tigrfam_name,cluster_num,MedianpI,LQpI,UQpI,MedianLength,LQLenths,UQLenths,Ecoli_host,Enterobacter_host
2,GCF_001041815.1_00006,150,"{Overcome classical restriction gp0.3: 102, hy...",MSNVTYHSLYLSAKQGLINVIRDNDLRDYDEACDYVHEVADIYVPH...,phrog_2568,ocr-like anti-restriction,"moron, auxiliary metabolic gene and host takeover",ocr,PF08684.13,NaN,...,NaN,3,4.050,4.0500,4.0500,117.0,114.00,121.00,True,True
4,GCA_009672325.1_00007,144,{hypothetical protein: 144},MIYTNEPANIFYMYVSAARADRPDVVNLARHNQLKRDIMNGLGMYG...,phrog_2226,SAM-dependent methyltransferase,other,NaN,NaN,NaN,...,NaN,5,5.515,5.2200,6.8200,152.0,152.00,155.00,True,True
9,GCA_021863795.1_00008,53,{hypothetical protein: 53},MTTNTSVITQEAGETVSIMSFREEVALEIRSHLDNIGQSYLAVGKA...,phrog_4571,NaN,unknown function,NaN,NaN,NaN,...,NaN,10,4.920,4.8500,5.0200,297.0,294.00,304.00,True,True
30,GCA_009667685.1_00049,17,{hypothetical protein: 17},MKLIGKGAFTKAYLLDSGRVQLHTCCPIKESMAWGWFPDSDLFPKL...,phrog_11084,NaN,unknown function,NaN,NaN,NaN,...,NaN,31,5.630,5.4100,5.9400,174.0,172.00,174.00,True,True
39,GCF_000917035.1_00005,12,"{Protein kinase 0.7: 11, hypothetical protein: 1}",MKDTMHRLTALLHEVMVAADYRVNCSGMGCAGQDSWVSFEAMARGE...,"phrog_18305,phrog_2828,phrog_15752","kinase,serine-threonine kinase,protein kinase;...",other,NaN,NaN,NaN,...,NaN,40,5.475,5.3500,5.6800,244.0,244.00,244.00,True,True
58,GCA_021863865.1_00009,8,{hypothetical protein: 8},MQKLIDALNASVFTLRTKGKATIKAEVAMLTEAGVTCDELDTSSKA...,phrog_4571,NaN,unknown function,NaN,NaN,NaN,...,NaN,59,5.000,4.9800,5.0425,344.0,338.75,344.00,True,True
90,GCF_002743475.1_00004,4,{hypothetical protein: 4},MLNRIDVHFKLLSAEKDIYFKRTWTTEPSDYPFIMGINIDGTMYYP...,phrog_23478,NaN,unknown function,NaN,NaN,NaN,...,NaN,91,4.360,4.3200,4.4025,166.0,166.00,166.00,False,True
103,GCA_021863795.1_00004,4,{hypothetical protein: 4},MIAQTIKKAKYTVIASASRSNLTEAANIDRHIDAGFTLSALGYDRH...,phrog_2226,SAM-dependent methyltransferase,other,NaN,NaN,NaN,...,NaN,104,5.650,5.5200,5.7800,143.0,143.00,143.00,False,True
106,GCA_020488535.1_00041,4,{hypothetical protein: 4},MERNANAYYDLLSATVELFRERIEQDQLTEDNYHDALHEVVDGQVP...,phrog_2568,ocr-like anti-restriction,"moron, auxiliary metabolic gene and host takeover",ocr,PF08684.13,NaN,...,NaN,107,4.050,4.0500,4.0525,172.5,172.00,175.50,False,True
114,GCF_002958225.1_00005,3,{hypothetical protein: 3},MRNLERAMQLATLTLDNFSNWFEDEGGVSHFVMSKVKMDEGTSQCE...,"phrog_18383,phrog_23974",NaN,unknown function,NaN,NaN,NaN,...,NaN,115,4.620,4.5950,4.6550,108.0,108.00,108.00,False,True


Next goal is to find, **which gene lay near the Ocr or SAMase or at the place.**

We need to find neighbours.

**Find SAMase and Ocr neighbours**

In [27]:
clusters_ocr = set([3, 107, 135, 297])
clusters_samase = set([5, 33, 55, 104, 196, 238, 289, 308, 372, 503, 626])
clusters_ard = set([256, 382, 488, 583])

In [28]:
gff_data.head()

,seq_id,start,end,strand,ID,file_id,inference,locus_tag,product,cluster_num,...,pfam_annotation,pfam,e_c_number,Name,gene,cdd,cdd_name,tigr,tigrfam_name,conj_pl_domains
0,AY264775.1,901,1254,+,GCA_002600005.1_00001,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00001,Overcome classical restriction gp0.3,2,...,ocr,PF08684.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AY264775.1,1254,1409,+,GCA_002600005.1_00002,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00002,Gene 0.4 protein,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AY264775.1,1478,1615,+,GCA_002600005.1_00003,upstreams_with_clusters,ab initio prediction:Prodigal:2.6,GCA_002600005.1_00003,hypothetical protein,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AY264775.1,1612,2256,+,GCA_002600005.1_00004,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00004,Protein kinase 0.7,134,...,NaN,NaN,2.7.11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AY264775.1,2327,4978,+,GCA_002600005.1_00005,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00005,T7 RNA polymerase,0,...,"RNA_pol,RPOL_N","PF00940.22,PF14700.9",2.7.7.6,1,1,NaN,NaN,NaN,NaN,NaN


In [29]:
gff_data.cluster_num = gff_data.cluster_num.apply(lambda x: int(x) + 1)

In [30]:
gff_data.head()

,seq_id,start,end,strand,ID,file_id,inference,locus_tag,product,cluster_num,...,pfam_annotation,pfam,e_c_number,Name,gene,cdd,cdd_name,tigr,tigrfam_name,conj_pl_domains
0,AY264775.1,901,1254,+,GCA_002600005.1_00001,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00001,Overcome classical restriction gp0.3,3,...,ocr,PF08684.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AY264775.1,1254,1409,+,GCA_002600005.1_00002,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00002,Gene 0.4 protein,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AY264775.1,1478,1615,+,GCA_002600005.1_00003,upstreams_with_clusters,ab initio prediction:Prodigal:2.6,GCA_002600005.1_00003,hypothetical protein,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AY264775.1,1612,2256,+,GCA_002600005.1_00004,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00004,Protein kinase 0.7,135,...,NaN,NaN,2.7.11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AY264775.1,2327,4978,+,GCA_002600005.1_00005,upstreams_with_clusters,"ab initio prediction:Prodigal:2.6,similar to A...",GCA_002600005.1_00005,T7 RNA polymerase,1,...,"RNA_pol,RPOL_N","PF00940.22,PF14700.9",2.7.7.6,1,1,NaN,NaN,NaN,NaN,NaN


In [31]:
gff_data[gff_data['cluster_num'].isin(clusters_ocr)].index

Index([   0,    3,    5,    8,   14,   23,   52,   89,  124,  141,
       ...
       3038, 3045, 3058, 3063, 3082, 3089, 3103, 3110, 3138, 3167],
      dtype='int64', length=158)

In [32]:
def read_json(json_like: str) -> dict:
    """
    read json string, returns it as a (dict) object
    """
    # based on file structure:
    json_like = json_like.replace(" '", ' "')
    json_like = json_like.replace("{'", '{"')
    json_like = json_like.replace("':", '":')

    js_d = json.loads(json_like)
    return js_d


def define_clu_main_product(products: dict) -> str:
    """

    :param functions: (dict) contains of all functions of cluster
    :return: main_function (str) name of cluster's main functional product
    """
    main_function = None
    max_count = 0
    for product, count in products.items():
        if product != 'hypothetical protein' and count > max_count:
            main_function = product
            max_count = count
    if not main_function and ('hypothetical protein' in products.keys()):
        main_function = 'hypothetical protein'
    return main_function


def extract_cluster_functions_wide(wide_path: str) -> dict:
    """
    extract information about main function of each cluster (dict) {cluster_parent: (number of cluster, function)}
    :param wide_path:
    :return:
    """
    cluster_to_function = {}
    with open(wide_path, 'r') as tsv_file:
        for ind, line in enumerate(tsv_file):
            cluster = line.strip().split('\t')
            clu_parent = cluster[0]
            clu_functs = cluster[2]
            clu_funct_js = read_json(clu_functs)
            clu_main_func = define_clu_main_product(clu_funct_js)
            cluster_to_function[clu_parent] = (ind + 1, clu_main_func)
    return cluster_to_function


def extract_cluster_children_long(long_path: str) -> dict:
    """
    extract information about children in clusters, returns dict {child: parent}
    :param long_path: path to long file with results of upstream gene products clusters
    :return: (dict) {child: parent} in cluster
    """
    childs_to_parent = {}

    with open(long_path, 'r') as tsv_file:
        for line in tsv_file:
            cluster = line.strip().split('\t')
            clu_parent = cluster[0]
            clu_child = cluster[1]
            childs_to_parent[clu_child] = clu_parent

    return childs_to_parent


def read_upstream_gff(gff_path: str) -> dict:
    upstreams = OrderedDict()
    with open(gff_path, 'r') as gff_file:
        for line in gff_file:
            if not line.startswith('#'):
                orf_info = line.strip().split('\t')
                chrom = orf_info[0]
                if chrom not in upstreams.keys():
                    upstreams[chrom] = [orf_info]
                else:
                    upstreams[chrom].append(orf_info)
    return upstreams


def add_cluster_information(upstreams: dict, clu_children: dict, clu_products: dict) -> dict:
    """
    add information about cluster for each cds in gff dict object:
    {genome: [[protein_1 gff], [protein_2 gff]]} -> add_cluster_information ->
    -> {genome: [[protein_1 gff, (num_1, function_1)],
                 [protein_2 gff, (num_2, function_2)]]
        }
    """
    upstreams_mod = upstreams.copy()

    for genome, cdss in upstreams.items():
        for ind, cds in enumerate(cdss):
            attribute = cds[8]
            attribute_list = attribute.split(';')
            cds_id = attribute_list[0].split('=')[1]
            parent = clu_children[cds_id]
            function = clu_products[parent]
            upstreams_mod[genome][ind].append(function)
    return upstreams_mod


def _find_numclu_one(clu_products: dict, keyword: str) -> set:
    """
    :param clu_products (tuple): (num_clu, main_product)
    :param keyword (str): product name, needed to find and exclude
    :return: (set) of cluster numbers to exclude
    """
    tabu_clu_num = set()
    for num, func in clu_products.values():
        if re.findall(keyword, func, re.IGNORECASE):
            tabu_clu_num.add(num)
    return tabu_clu_num


def find_numclu_to_filter(clu_products: dict, keywords: set) -> set:
    """
    uses `_find_numclu_one` for set of keywords
    :param clu_products: clu_products (tuple): (num_clu, main_product)
    :param keywords (set): of product names, needed to find and exclude
    :return: (set) of cluster numbers to exclude
    """
    tabu_clu_nums = set()
    for keyword in keywords:
        tabu_clu_nums = tabu_clu_nums | _find_numclu_one(clu_products, keyword)
    return tabu_clu_nums


def filter_with_conditions(upstream_mod):

    if sum(upstream_mod['strand'] == '-') == len(upstream_mod):  # potential error place
        select_indices = list(np.where(upstream_mod['cluster_in_tabu'])[0])
        target_index = select_indices[0]
    elif sum(upstream_mod['strand'] == '+') == len(upstream_mod):
        select_indices = list(np.where(upstream_mod['cluster_in_tabu'])[0])
        target_index = select_indices[-1]
    else:
        chormosome = upstream_mod.chromosome.unique()[0]
        raise ValueError(f'Non-unique strand for {chormosome}')

    start = upstream_mod.iloc[target_index]['start']
    end = upstream_mod.iloc[target_index]['end']

    if sum(upstream_mod['strand'] == '-') == len(upstream_mod):
        condition = (upstream_mod['end'] <= end) | (upstream_mod['start'] - 5000 >= start)
    else:
        condition = (upstream_mod['end'] >= end) | (upstream_mod['start'] + 5000 <= start)

    upstream_mod = upstream_mod[condition]
    return upstream_mod


def filter_within_one_genome(upstream: list, tabu_clusters: set) -> list:
    """
    finds tabu genes in one genome and returns the resulting list without it's upstream region
    :param upstream (list): part of upstream.gff for one region:
    {genome (list): [[protein_1 gff, (num_1, function_1)],
                 [protein_2 gff, (num_2, function_2)]]
        }
    :param tabu_clu_nums (set): num of tabu
    :return: upstream_mod (list) with removed tabu genes and their upstream
    """
    colnames_ = ['chromosome', 'annotator', 'type', 'start', 'end',
                 'something', 'strand', 'somthing_else', 'attribute', 'cluster']

    upstream_df = pd.DataFrame(upstream, columns=colnames_)  # list to dataframe
    # numbers: str to int
    upstream_df['start'] = list(map(lambda x: int(x), upstream_df['start']))
    upstream_df['end'] = list(map(lambda x: int(x), upstream_df['end']))
    # column: cluster number:
    upstream_df['cluster_num'] = list(map(lambda x: x[0], upstream_df['cluster']))
    # column: is tabu cluster?
    upstream_df['cluster_in_tabu'] = list(map(lambda x: x in tabu_clusters, upstream_df['cluster_num']))

    if sum(upstream_df['cluster_in_tabu']) > 0:
        upstream_df = filter_with_conditions(upstream_df)
    upstream_mod = upstream_df.iloc[:, :10].values.tolist()
    return upstream_mod


def make_gff_with_clusters(path_long: str, path_wide: str, path_upstream: str):
    children_to_clu = extract_cluster_children_long(path_long)
    parent_to_func = extract_cluster_functions_wide(path_wide)
    upstreams = read_upstream_gff(path_upstream)
    ups_with_clu_info = add_cluster_information(upstreams=upstreams,
                                                clu_children=children_to_clu,
                                                clu_products=parent_to_func)
    upstreams_mod = []
    for upstream in ups_with_clu_info.values():
        upstreams_mod.append(upstream)
    return upstreams_mod

In [33]:
# p_wide = 'results/upstream_proteins_clu_wide_seq_sorted_prokka_domains_pi_length_host.tsv'
# p_l = 'results/upstream_proteins_clu_long_prokka.tsv'
# gff_file 
path_wide_ = os.path.join(res_dir, 'upstream_proteins_clu_wide_seq_sorted_prokka.tsv')
path_long_ = os.path.join(res_dir, 'upstream_proteins_clu_long_prokka.tsv')
upstreams = make_gff_with_clusters(path_long=path_long_, path_wide=path_wide_, path_upstream=gff_file)

In [34]:
pd.DataFrame(upstreams[1])

,0,1,2,3,4,5,6,7,8,9
0,AY264776.1,Prodigal:2.6,CDS,901,1254,.,+,0,ID=GCA_002600025.1_00001;file_id=upstreams_wit...,"(3, Overcome classical restriction gp0.3)"
1,AY264776.1,Prodigal:2.6,CDS,1254,1409,.,+,0,ID=GCA_002600025.1_00002;file_id=upstreams_wit...,"(13, Gene 0.4 protein)"
2,AY264776.1,Prodigal:2.6,CDS,1557,4208,.,+,0,ID=GCA_002600025.1_00003;Name=1;file_id=upstre...,"(1, T7 RNA polymerase)"


In [35]:
colnames_ = ['chromosome', 'annotator', 'type', 'start', 'end',
             'something', 'strand', 'somthing_else', 'attribute', 'cluster']


def find_right_neighbour(upstream, gene, index):
    strand = gene[6]
    if strand == '+':
        if index != len(upstream) - 1:
            right_neighbour = upstream[index + 1]
            gene_end = int(gene[4])
            right_neighbour_start = int(right_neighbour[3])
            if gene_end + 5000 > right_neighbour_start:
                return right_neighbour[9][0]
            else:
                return None
        else:
            right_neighbour = upstream[0]
            gene_start = int(gene[3])
            right_neighbour_end = int(right_neighbour[4])
            if gene_start - 5000 > right_neighbour_end: 
                return right_neighbour[9][0]
            else:
                return None
    else:
        if index != 0:
            right_neighbour = upstream[index - 1]
            gene_start = int(gene[3])
            right_neighbour_end = int(right_neighbour[4])
            if gene_start - 5000 < right_neighbour_end:
                return right_neighbour[9][0]
            else:
                return None
        else:
            right_neighbour = upstream[-1]
            gene_end = int(gene[4])
            right_neighbour_start = int(right_neighbour[3])
            if gene_end + 5000 < right_neighbour_start:
                 return right_neighbour[9][0]
            else:
                return None
        

right_neighbours = defaultdict(list)

for upstream in upstreams:
    for i, gene in enumerate(upstream):
        cluster = gene[9][0]
        right_neighbours[cluster].append(find_right_neighbour(upstream, gene, i))

In [36]:
def find_left_neighbour(upstream, gene, index):
    strand = gene[6]
    if strand == '-':
        if index != len(upstream) - 1:
            left_neighbour = upstream[index + 1]
            gene_end = int(gene[4])
            left_neighbour_start = int(left_neighbour[3])
            if gene_end + 5000 > left_neighbour_start:
                return left_neighbour[9][0]
            else:
                return None
        else:
            left_neighbour = upstream[0]
            gene_start = int(gene[3])
            left_neighbour_end = int(left_neighbour[4])
            if gene_start - 5000 > left_neighbour_end: 
                return left_neighbour[9][0]
            else:
                return None
    else:
        if index != 0:
            left_neighbour = upstream[index - 1]
            gene_start = int(gene[3])
            left_neighbour_end = int(left_neighbour[4])
            if gene_start - 5000 < left_neighbour_end:
                return left_neighbour[9][0]
            else:
                return None
        else:
            left_neighbour = upstream[-1]
            gene_end = int(gene[4])
            left_neighbour_start = int(left_neighbour[3])
            if gene_end + 5000 < left_neighbour_start:
                 return left_neighbour[9][0]
            else:
                return None
        

left_neighbours = defaultdict(list)

for upstream in upstreams:
    for i, gene in enumerate(upstream):
        cluster = gene[9][0]
        left_neighbours[cluster].append(find_left_neighbour(upstream, gene, i))

In [37]:
def extract_neighbours(neighbours_dict: dict, clu_nums: str) -> set:
    right_neighbours = set()
    for clu in clu_nums:
        for neighbour in neighbours_dict[clu]:  # 0-based
            right_neighbours.add(neighbour)  # 1-based
    return right_neighbours


ocr_neighbours = extract_neighbours(right_neighbours, clusters_ocr)
samase_neighbours = extract_neighbours(right_neighbours, clusters_samase)
ard_neighbours = extract_neighbours(right_neighbours, clusters_ard)

In [38]:
len(ocr_neighbours)

21

In [39]:
def get_common_neighbours_one_clu(neigbours_dict: dict, target_neighbours: set, clu: int) -> set:
    neighbours_clu = set(neigbours_dict[clu])
    common_ns = target_neighbours & neighbours_clu
    return common_ns


def get_common_neighbours(neigbours_dict: dict, target_neighbours: set, n: int) -> dict:
    common_neighbours = {}
    for clu in range(1, n):
        common = get_common_neighbours_one_clu(neigbours_dict, target_neighbours, clu)
        common_neighbours[clu] = common
    return common_neighbours

In [40]:
n = len(df_agg)+2

ocr_neighbours = extract_neighbours(right_neighbours, clusters_ocr)
samase_neighbours = extract_neighbours(right_neighbours, clusters_samase)
ard_neighbours = extract_neighbours(right_neighbours, clusters_ard)
print(f'Common right neighbours for Ocr and SAMase: {ocr_neighbours & samase_neighbours}')
print(f'Common right neighbours for Ocr and ArdA/ArdcN: {ocr_neighbours & ard_neighbours}')
print(f'Common right neighbours for SAMase and ArdA/ArdcN: {samase_neighbours & ard_neighbours}')

res_table['common_ocr_right_neighbour'] = res_table['cluster_num'].apply(lambda x: 
                                                                         get_common_neighbours_one_clu(right_neighbours, 
                                                                                                       ocr_neighbours, x))
res_table['common_samase_right_neighbour'] = res_table['cluster_num'].apply(lambda x: 
                                                                         get_common_neighbours_one_clu(right_neighbours, 
                                                                                                       samase_neighbours, x))
res_table['common_ard_right_neighbour'] = res_table['cluster_num'].apply(lambda x: 
                                                                         get_common_neighbours_one_clu(right_neighbours, 
                                                                                                       ard_neighbours, x))


Common right neighbours for Ocr and SAMase: {32, 19, 21, 6}
Common right neighbours for Ocr and ArdA/ArdcN: {139}
Common right neighbours for SAMase and ArdA/ArdcN: set()


In [41]:
ocr_neighbours = extract_neighbours(left_neighbours, clusters_ocr)
samase_neighbours = extract_neighbours(left_neighbours, clusters_samase)
ard_neighbours = extract_neighbours(left_neighbours, clusters_ard)

print(f'Common left neighbours for Ocr and SAMase: {ocr_neighbours & samase_neighbours}')
print(f'Common left neighbours for Ocr and ArdA/ArdcN: {ocr_neighbours & ard_neighbours}')
print(f'Common left neighbours for SAMase and ArdA/ArdcN: {samase_neighbours & ard_neighbours}')

res_table['common_ocr_left_neighbour'] = res_table['cluster_num'].apply(lambda x: 
                                                                         get_common_neighbours_one_clu(left_neighbours, 
                                                                                                       ocr_neighbours, x))
res_table['common_samase_left_neighbour'] = res_table['cluster_num'].apply(lambda x: 
                                                                         get_common_neighbours_one_clu(left_neighbours, 
                                                                                                       samase_neighbours, x))
res_table['common_ard_left_neighbour'] = res_table['cluster_num'].apply(lambda x: 
                                                                         get_common_neighbours_one_clu(left_neighbours, 
                                                                                                       ard_neighbours, x))


Common left neighbours for Ocr and SAMase: {41, 18, None}
Common left neighbours for Ocr and ArdA/ArdcN: {None, 14}
Common left neighbours for SAMase and ArdA/ArdcN: {None}


In [42]:
res_table.head()

,clu_parent,count,products,sequence,phrOG,phrOG_annotation,phrOG_category,pfam_annotation,pfam,conj_pl_domains,...,LQLenths,UQLenths,Ecoli_host,Enterobacter_host,common_ocr_right_neighbour,common_samase_right_neighbour,common_ard_right_neighbour,common_ocr_left_neighbour,common_samase_left_neighbour,common_ard_left_neighbour
0,GCF_001470995.1_00023,354,{T7 RNA polymerase: 354},MEYNPLTELASLYGEDLAAEQLRLEAEAYSLGEKRFMEAMEFKAET...,"phrog_20599,phrog_414",RNA polymerase,"DNA, RNA and nucleotide metabolism","RNA_pol,RPOL_N","PF00940.22,PF14700.9",NaN,...,883.0,906.0,True,True,{},{},{},{14},{3},{14}
1,GCA_025085155.1_00014,190,"{Protein kinase 0.7: 183, hypothetical protein...",MYAHFQSLLKAIRELPINELDKRQPMLVDLLAQIVNHETQDGAITA...,"phrog_18305,phrog_2828,phrog_15752","kinase,serine-threonine kinase,protein kinase;...",other,NaN,NaN,NaN,...,337.0,369.0,True,True,{1},{},{},"{19, 14}",{3},{14}
2,GCF_001041815.1_00006,150,"{Overcome classical restriction gp0.3: 102, hy...",MSNVTYHSLYLSAKQGLINVIRDNDLRDYDEACDYVHEVADIYVPH...,phrog_2568,ocr-like anti-restriction,"moron, auxiliary metabolic gene and host takeover",ocr,PF08684.13,NaN,...,114.0,121.0,True,True,"{1, 2, 5, 6, 8, 10, 12, 13, 16, 19, 21, 32, 56...","{32, 19, 21, 6}",{},"{5, 647, 11, 14, 18, 22, 534, 29, 681, 41, 45,...","{41, 18, None}","{None, 14}"
3,GCA_023716895.1_00035,149,{hypothetical protein: 149},METSCRLPHGAGRSCTGMDGDAMMFTAFIVFMYLLIVLYFLKDFRK...,phrog_2385,NaN,unknown function,NaN,NaN,NaN,...,65.0,65.0,True,True,"{1, 2, 13}",{},{},"{18, 11, 5}","{18, 77}",{}
4,GCA_009672325.1_00007,144,{hypothetical protein: 144},MIYTNEPANIFYMYVSAARADRPDVVNLARHNQLKRDIMNGLGMYG...,phrog_2226,SAM-dependent methyltransferase,other,NaN,NaN,NaN,...,152.0,155.0,True,True,"{32, 19, 21, 6}","{512, 3, 4, 645, 6, 11, 14, 18, 19, 21, 24, 32...",{},"{41, 18, None}","{3, 230, 41, 265, 141, 18, None, 61}",{None}


In [43]:
res_table.to_csv(output_file, sep='\t', index=False)